# LCM2023 - Poster session, 5-8 September 2023
## Retrofitting of Italian vernacular built stock with bio-based building materials: a viable solution to mitigate climate change effects?
Example of the brightway code used to compute the LCA results included in the contribution to the LCM conference.

In [ ]:
Project_name="BIOED" 
ei_version="3.9.1"
ei_type="cut-off"
ei_name="Ecoinvent "+ei_version+" "+ei_type+" "+Project_name
ei_name

## Library used

In [ ]:
# brightway 25
import bw2analyzer as ba
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import matrix_utils as mu
import bw_processing as bp
# import bw_temporalis

import pandas as pd
import os 
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px

path_name = r'C:\Users\niky2\OneDrive - Politecnico di Milano\Dottorato\Progetti\01-Paper climate\Aggiornamento LCA'
os.chdir(path_name)

In [ ]:
bd.projects.set_current('BIOED')
  

In [ ]:
bd.projects.current

In [ ]:
bi.bw2setup() # se il project esiste già, questo comando non fa nulla
# se il progetto non esiste, bw2 crea il biosphere database, che è il database dei flussi elementari. Crea anche gli
# LCIA methods di base. Si possono poi importare altri flussi elementari o LCIA methods.
# il biosphere database è creato a partire da file presenti all'interno dei packages di bw2, che vengono richiamati quando serve

In [ ]:
list(bd.databases)

## Ecoinvent 3.9.1

In [ ]:
if len(bd.databases)==1: # eseguo questo comando solo se ho solo il biosphere database (database dei flussi elementari). Procedo a importare un nuovo database io, a partire da ecoinvent 3.8 cutoff 
    # Nel comando qui sotto, dovrai mettere l'indirizzo del tuo pc in cui hai salvato ecoinvent in formato ecospold. 
    # Attenzione a unzippare il file ecospold e ad arrivare alla cartella datasets nell'indirizzo
    fpei = r'D:\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets'
    # i comandi qui sotto servono a elaborare il database e infine "scriverlo" nel project
    ei = bi.SingleOutputEcospold2Importer(fpei, ei_name) # creazione dell'oggetto 'Ecoinvent 3.8 cut-off OWC'.
    ei.apply_strategies() # elaborazione del database
    ei.statistics() # statistiche finali (eventuali errori vengono mostrati qui)
    ei.write_database() # scrittura del database

## Implemented function:

In [ ]:
def new_act_diff_location(database, activity_id, locations, new_name, project_name=Project_name):
    ''' This function will create a copy of an activity, identified with activity.id, and will modify the copied activity by 
    substituting the technosphere processes with the preferable location indicated by the parameter locations.
    '''
    # is the modified process already in the database?
    if (len([act for act in database if act['name']==new_name])==0):
        # find activity to modify
        act_origin = [act for act in database if act.id==activity_id][0]
        act_new = act_origin.copy()
        act_new['name'] = new_name
        act_new['location'] = project_name
        act_new.save()
        
        for exc in act_new.technosphere():
            exc.delete()
            exc.save()
            act_new.save()
            
        # If GLO or RoW were not considered, add it at the end
        if 'RoW' not in locations:
            locations.append('RoW')
        if 'GLO' not in locations:
            locations.append('GLO')
        
        locations.reverse()
        
        for exc in act_origin.technosphere():
            for loc in locations:
            # Search for an exact match of the reference product with specified location
                for activity in database:
                    if activity['reference product'] == exc['name'] and loc in activity['location'] and activity['name'] == exc.input['name']:
                        new_edge = activity
                
             
            act_new.new_edge(
                            input=new_edge.key,
                            name=new_edge['name'],
                            amount=exc.amount,
                            type="technosphere",
                        ).save()
            new_edge.save()             
        
        return act_new
    
    print('The modified activity is already present in the database!!')
    return [act for act in database if act['name']==new_name][0]
    

In [ ]:
def new_activity(database, activity_name, activity_code, activity_unit, activity_reference, exchanges, project_name=Project_name, bio = bd.Database('biosphere3')):
    '''This function will create a new activity in the given database, adding in input the given exchanges'''
    
    # is the modified process already in the database?
    if (len([act for act in database if act['name']==activity_name and act['location']==project_name])==0):
        print('####################################################################################################')
        print(f'Creating the activity: {activity_name}....')
        proj_Database=database.new_node(code=activity_code,
                                                         name=activity_name, # nome
                                                         location=project_name, 
                                                         type='process', 
                                                         unit=activity_unit) # unità di misura del reference flow del processo
        # e lo salvo poi nel database (bisogna sempre ricordarsi di salvare, altrimenti le modifiche)
        proj_Database.save()
        # bw2 usa il seguente vocabolario, rispetto a OpenLCA: activity=process e exchange=flow
        # Qui sotto cerco uno specifico processo (activity), nel database 'ecoinvent 3.6 LCE' indicando che 'Polymer transport - LCE' sia nel nome di tale processo.
        # Ovvero, sto cercando il processo che ho appena creato nelle righe sopra.
        new_act=[act for act in database if act['name']==activity_name and act['location']==project_name][0]
        # Aggiungo il nome del reference flow nelle info di questo processo
        new_act['reference product']=activity_reference
        # e poi salvo.
        new_act.save()
        
        print(f'Inseriting the inputs of {activity_name}:')
        for j in range(len(exchanges)):
            if exchanges['code'][j]==new_act['code']:
                if exchanges['location'][j] == 'biosphere':
                    new_input=[ef for ef in bio if exchanges['process'][j] in ef['name']][0]
                    
                    new_exc=new_act.new_edge(input=new_input.key, # Questo è il provider
                               amount=exchanges['amount'][j], # quantità
                               type='biosphere')
                    new_exc.save()
                    new_act.save()
                    print(f"Edge: {new_input['name']};    Amount: {exchanges['amount'][j]}")
                    
                else: 
                    new_input=[act for act in ei if act['name'] == exchanges['process'][j]
                                if exchanges['location'][j] in act['location']][0]

                    new_exc=new_act.new_edge(input=new_input.key, # Questo è il provider
                                   amount=exchanges['amount'][j], # quantità
                                   unit=new_input["unit"], # unità di misura
                                   name=new_input['reference product'], # nome del flusso
                                   type='technosphere')
                    new_exc.save()
                    new_act.save()        
                    print(f"Edge: {new_input['reference product']};    Amount: {exchanges['amount'][j]}")
        
        
    else:
        print('______________________________________________________________________________________________________')
        print(f'The activity: {activity_name} is already present in the database, no need to create a new one!!')
    
    

## Materials LCI

In [ ]:
ei = bd.Database(ei_name)

In [ ]:
bio = bd.Database('biosphere3')

In [ ]:
[act for act in ei if 'BIOED' in act['location']]

## creation of proxy activities for Italy

### lime mortar production, IT

for example to create the activity 'lime mortar production' located in Italy, we started from the activity located in Switzerland and chainging the input with the corresponding activities, coherently selected for the Italian case.  
To search the locations, we considered the presence of activities with the following hierarchical order:  
- IT;
- Europe without Switzerland;
- RER;
- RoW;
- GLO.

Creation of the activity 'lime mortar production, IT' from the correspondent activity in Switzerland but with different location for the input.

In [ ]:
lime_mortar_CH = [act for act in ei if 'lime mortar production' in act['name'] and 'CH' in act['location']][0]
lime_mortar_CH.id

In [ ]:
LM_IT_node = new_act_diff_location(ei, lime_mortar_CH.id, ['IT', 'Europe', 'RER'], "lime mortar production, IT", Project_name)

In [ ]:
[exc for exc in LM_IT_node.technosphere()]

In [ ]:
[exc for exc in lime_mortar_CH.technosphere()]

### plaster mixing, IT 

In [ ]:
plaster_mixing_CH = [act for act in ei if 'plaster mixing' in act['name'] and 'CH' in act['location']][0]

In [ ]:
plaster_mixing_CH.id

In [ ]:
PM_IT_node.delete()

In [ ]:
PM_IT_node = new_act_diff_location(ei, plaster_mixing_CH.id, ['IT', 'Europe', 'RER'], "plaster mixing, IT", Project_name)

In [ ]:
[exc for exc in PM_IT_node.technosphere()]

## Materials LCI from BIOED_import.xlsx

In [ ]:
activities_df = pd.read_excel(r"BIOED_import.xlsx", "act")
activities_df

In [ ]:
exchanges_df = pd.read_excel(r"BIOED_import.xlsx", "exc")
exchanges_df

In [ ]:
for i in range(len(activities_df)):
    new_activity(ei, activities_df['name'][i], activities_df['code'][i], 
                 activities_df['unit'][i], activities_df['reference product'][i], exchanges_df)